In [ ]:

!git clone https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
!git reset --hard
!git pull -f https://github.com/MaastrichtU-IDS/LMKBC-2023.git

In [ ]:
import os
os.chdir('LMKBC-2023/')

In [ ]:
!pip install accelerate -U
#!pip install transformers

In [ ]:
!python src/run.py



   python src/tr_model.py    --train_batch_size 128 --gpu 0   --train_epoch 30 --learning_rate 3e-5  --model_load_dir prajjwal1/bert-tiny --model_save_dir /workspace/persistent/LMKBC-2023/bin/train_full/token_redefine/prajjwal1/bert-tiny --model_best_dir  /workspace/persistent/LMKBC-2023/bin/train_full/token_redefine/prajjwal1/bert-tiny/best_ckpt
    
    
True
max_sentence_length 40
train dataset size: 6925
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|                                                  | 0/1650 [00:00<?, ?it/s]You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method

In [ ]:
import torch

print(torch.cuda.is_available())